<a href="https://colab.research.google.com/github/monoramasn/Speech_fairness/blob/main/sl_whisper_large_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install git+https://github.com/huggingface/transformers
!pip install librosa
!pip install evaluate>=0.30
!pip install jiwer
!pip install gradio

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-1vmhx3sh
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-1vmhx3sh
  Resolved https://github.com/huggingface/transformers to commit 510270af34994c24f2f8792d2afd74114c70d736
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.36.0.dev0-py3-none-any.whl size=8076097 sha256=c6c7c8edebe8be4a032699f3941658e14edac31f3111564276839d96e9fa0b58
  Stored in directory: /tmp/pip-ephem-wheel-cache-d3tf7utj/wheels/c0/14/d6/6c9a5582d2ac191ec0a483be151a4495fe1eb2a6706ca49f1b
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
import torch
from transformers import AutoFeatureExtractor, WhisperModel
from datasets import load_dataset

In [4]:
import os
import numpy as np

try:
    import tensorflow  # required in Colab to avoid protobuf compatibility issues
except ImportError:
    pass

import torch
import pandas as pd
import torchaudio

from tqdm.notebook import tqdm

In [5]:
voxpopuli_data = load_dataset("facebook/voxpopuli", "sl", split="validation")
#column_name = "audio, normalized_text"

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [6]:
voxpopuli_data

Dataset({
    features: ['audio_id', 'language', 'audio', 'raw_text', 'normalized_text', 'gender', 'speaker_id', 'is_gold_transcript', 'accent'],
    num_rows: 889
})

In [7]:
!pip install openai-whisper
import whisper
#model = whisper.load_model("base")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.6/798.6 kB 9.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 23.4 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20231117-py3-none-any.whl size=801356 sha256=453f950e10f6b9b1a6368db2400ed44173f03f57d33a4ec9199f81a53312c067
  Stored in directory: /root/.cache/pip/wheels/d0/85/e1/9361b4cbea7dd4b7f6702fa4c3afc94877952eeb2b62f45f56
Successfully built openai-whisper
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires openai, which is not installed.


In [8]:
model = whisper.load_model("large")

100%|█████████████████████████████████████| 2.88G/2.88G [00:50<00:00, 61.1MiB/s]


In [9]:
from transformers import WhisperForConditionalGeneration, WhisperFeatureExtractor,WhisperTokenizer
from datasets import load_dataset
import torch
from tqdm import tqdm
from jiwer import wer

In [10]:
Gender=voxpopuli_data['gender']
Audio=voxpopuli_data['audio']
text=voxpopuli_data['normalized_text']
Gender=pd.DataFrame(Gender)
Audio=pd.DataFrame(Audio)
text=pd.DataFrame(text)

In [11]:
text

,0
0,ko gre za medije v srbiji velja da ustrezna za...
1,do uresničitve tega cilja pa bo še dolga pot.
2,se pravi gre za velik obseg in prav je da na t...
3,krčenju deževnih gozdov sem podprl saj se v pr...
4,prav seveda je da soočimo različne argumente i...
...,...
884,in temu so ta sredstva namenjena.
885,prav tako smo poudarili da ko gre za efsi v os...
886,gre za poseg v digniteto posameznika poseg v t...
887,spoštovana visoka predstavnica spoštovani kolegi.


In [12]:
Gender.columns=['Gender']
text.columns=['Transcript']

In [14]:
file=pd.concat([Gender, text, Audio],axis=1)
file

,Gender,Transcript,path,array,sampling_rate
0,male,ko gre za medije v srbiji velja da ustrezna za...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.159332275390625, -0.184112548828125, -0.20...",16000
1,male,do uresničitve tega cilja pa bo še dolga pot.,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.000213623046875, 0.000274658203125, 0.0004...",16000
2,male,se pravi gre za velik obseg in prav je da na t...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.002227783203125, -0.019012451171875, -0.033...",16000
3,male,krčenju deževnih gozdov sem podprl saj se v pr...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.000335693359375, -0.000701904296875, -0.00...",16000
4,male,prav seveda je da soočimo različne argumente i...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.000152587890625, -9.1552734375e-05, -0.0002...",16000
...,...,...,...,...,...
884,male,in temu so ta sredstva namenjena.,/root/.cache/huggingface/datasets/downloads/ex...,"[0.000457763671875, 0.0003662109375, 0.0003662...",16000
885,female,prav tako smo poudarili da ko gre za efsi v os...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.00262451171875, -0.009429931640625, 0.00576...",16000
886,male,gre za poseg v digniteto posameznika poseg v t...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.058563232421875, 0.065948486328125, 0.07632...",16000
887,male,spoštovana visoka predstavnica spoštovani kolegi.,/root/.cache/huggingface/datasets/downloads/ex...,"[-3.0517578125e-05, -0.000213623046875, 3.0517...",16000


In [15]:
male_samples=file.loc[file['Gender'] == 'male']
male_samples

,Gender,Transcript,path,array,sampling_rate
0,male,ko gre za medije v srbiji velja da ustrezna za...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.159332275390625, -0.184112548828125, -0.20...",16000
1,male,do uresničitve tega cilja pa bo še dolga pot.,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.000213623046875, 0.000274658203125, 0.0004...",16000
2,male,se pravi gre za velik obseg in prav je da na t...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.002227783203125, -0.019012451171875, -0.033...",16000
3,male,krčenju deževnih gozdov sem podprl saj se v pr...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.000335693359375, -0.000701904296875, -0.00...",16000
4,male,prav seveda je da soočimo različne argumente i...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.000152587890625, -9.1552734375e-05, -0.0002...",16000
...,...,...,...,...,...
882,male,jasno je tudi sporočilo da je potrebno spoštov...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.000946044921875, -0.00189208984375, -0.003...",16000
884,male,in temu so ta sredstva namenjena.,/root/.cache/huggingface/datasets/downloads/ex...,"[0.000457763671875, 0.0003662109375, 0.0003662...",16000
886,male,gre za poseg v digniteto posameznika poseg v t...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.058563232421875, 0.065948486328125, 0.07632...",16000
887,male,spoštovana visoka predstavnica spoštovani kolegi.,/root/.cache/huggingface/datasets/downloads/ex...,"[-3.0517578125e-05, -0.000213623046875, 3.0517...",16000


In [16]:
female_samples=file.loc[file['Gender'] == 'female']
female_samples

,Gender,Transcript,path,array,sampling_rate
6,female,želimo si zdravega posameznika fizično zdraveg...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.001495361328125, -0.002655029296875, -0.004...",16000
7,female,države razsodbe sodišč je treba spoštovati. ji...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.018035888671875, -0.01031494140625, 0.0296...",16000
11,female,in to je nujno kajti s tem bodo tudi ljudje ob...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.01763916015625, 0.0198974609375, 0.01605224...",16000
18,female,do teh zavrženih dejanj moramo imeti ničelno t...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.00042724609375, -0.000152587890625, 0.0, 0....",16000
19,female,pred letom dni smo imeli prve sestanke in zato...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.003448486328125, 0.00628662109375, 0.004028...",16000
...,...,...,...,...,...
873,female,dva tisoč dvanajst takrat je bila dana zaveza ...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.052001953125, -0.051116943359375, -0.03588...",16000
874,female,na ravni evropske unije potrebujemo močno in n...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.004791259765625, -0.00030517578125, 0.0050...",16000
877,female,zato bi se morali vprašati kaj vendarle delamo...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.0067138671875, 0.02056884765625, 0.01608276...",16000
883,female,poleg sistematičnega preverjanja identitete uv...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.002410888671875, -0.0015869140625, -0.0085...",16000


In [18]:
def transcribe_audio(file_path):
    result = model.transcribe(file_path, language="sk")
    return result["text"]

In [19]:
def calculate_wer(ground_truths, hypotheses):
    return wer(ground_truths, hypotheses)

In [20]:
total_wer = 0
num_sample = 0

for index,row in male_samples.iterrows():
    audio_path=row['path']
    ground_truth_text = row['Transcript']
    transcription = transcribe_audio(audio_path)
    # Compute metrics
    sample_wer = calculate_wer(ground_truth_text, transcription)

    total_wer += sample_wer
    num_sample += 1

    print(f"Processed {num_sample} files, WER: {sample_wer}")

# Calculate average metrics
average_wer = total_wer / num_sample

print(f"Average WER: {average_wer}")

Processed 1 files, WER: 1.0
Processed 2 files, WER: 0.5555555555555556
Processed 3 files, WER: 0.9230769230769231
Processed 4 files, WER: 1.0
Processed 5 files, WER: 0.75
Processed 6 files, WER: 1.0
Processed 7 files, WER: 0.6923076923076923
Processed 8 files, WER: 0.8
Processed 9 files, WER: 0.8
Processed 10 files, WER: 0.8461538461538461
Processed 11 files, WER: 0.7058823529411765
Processed 12 files, WER: 0.85
Processed 13 files, WER: 0.46153846153846156
Processed 14 files, WER: 0.7619047619047619
Processed 15 files, WER: 1.0
Processed 16 files, WER: 0.875
Processed 17 files, WER: 0.85
Processed 18 files, WER: 0.75
Processed 19 files, WER: 0.5454545454545454
Processed 20 files, WER: 0.631578947368421
Processed 21 files, WER: 0.6956521739130435
Processed 22 files, WER: 0.9166666666666666
Processed 23 files, WER: 0.7586206896551724
Processed 24 files, WER: 0.868421052631579
Processed 25 files, WER: 1.0
Processed 26 files, WER: 0.8214285714285714
Processed 27 files, WER: 0.7
Processed 2

In [21]:
total_wer = 0
num_sample = 0

for index,row in female_samples.iterrows():
    audio_path=row['path']
    ground_truth_text = row['Transcript']
    transcription = transcribe_audio(audio_path)
    # Compute metrics
    sample_wer = calculate_wer(ground_truth_text, transcription)

    total_wer += sample_wer
    num_sample += 1

    print(f"Processed {num_sample} files, WER: {sample_wer}")

# Calculate average metrics
average_wer = total_wer / num_sample

print(f"Average WER: {average_wer}")

Processed 1 files, WER: 0.6428571428571429
Processed 2 files, WER: 1.0
Processed 3 files, WER: 0.6923076923076923
Processed 4 files, WER: 0.75
Processed 5 files, WER: 0.6785714285714286
Processed 6 files, WER: 0.6176470588235294
Processed 7 files, WER: 0.46153846153846156
Processed 8 files, WER: 0.5757575757575758
Processed 9 files, WER: 0.5789473684210527
Processed 10 files, WER: 1.0
Processed 11 files, WER: 0.7857142857142857
Processed 12 files, WER: 0.45454545454545453
Processed 13 files, WER: 0.5714285714285714
Processed 14 files, WER: 1.0
Processed 15 files, WER: 1.0
Processed 16 files, WER: 0.6428571428571429
Processed 17 files, WER: 0.68
Processed 18 files, WER: 0.7368421052631579
Processed 19 files, WER: 0.42857142857142855
Processed 20 files, WER: 0.8888888888888888
Processed 21 files, WER: 0.5882352941176471
Processed 22 files, WER: 0.8571428571428571
Processed 23 files, WER: 1.0714285714285714
Processed 24 files, WER: 0.5555555555555556
Processed 25 files, WER: 0.82352941176